# Generation of Synthetic Manuals

## Expert Graph and Node Analysis
Analyzes and displays the causal relationships between components.

In [7]:
import networkx as nx
import pandas as pd

G = nx.read_gml("factorybench/data/causrca_base/expert_graph/expert_graph.gml")

print(f"Nodes: {G.number_of_nodes()}")  # 92
print(f"Edges: {G.number_of_edges()}")  # 104

nodes_df = pd.read_csv("factorybench/data/causrca_base/expert_graph/all_nodes.csv")

print("\nColumns available in NODES:")
print(nodes_df.columns.tolist())

print("\nFirst few rows:")
print(nodes_df.head())

nodes_df['subsystem'] = nodes_df['path'].apply(lambda x: x.split('/')[1] if '/' in x else 'Unknown')

print("\n✅ Nodes by subsystem:")
subsystem_counts = nodes_df.groupby('subsystem').size()
print(subsystem_counts)


print("\n✅ Nodes by type:")
print(nodes_df.groupby('type').size())


subsystems = nodes_df['subsystem'].unique()
for subsystem in subsystems:
    print(f"\n{'='*60}")
    print(f"Subsystem: {subsystem}")
    print(f"{'='*60}")
    
    subsystem_nodes = nodes_df[nodes_df['subsystem'] == subsystem]
    
    # separate variables and alarms
    variables = subsystem_nodes[subsystem_nodes['type'] == 'Variable']
    alarms = subsystem_nodes[subsystem_nodes['type'] == 'Alarm']
    
    print(f"Variables ({len(variables)}):")
    for _, node in variables.head(10).iterrows():  # Solo primeras 10
        print(f"  - {node['label']}: {node.get('comment', 'N/A')[:50]}...")
    
    if len(variables) > 10:
        print(f"  ... and {len(variables) - 10} more")
    
    print(f"\nAlarms ({len(alarms)}):")
    for _, node in alarms.head(10).iterrows():  # Solo primeras 10
        print(f"  - {node['label']}: {node.get('comment', 'N/A')[:50]}...")
    
    if len(alarms) > 10:
        print(f"  ... and {len(alarms) - 10} more")

# 6. save nodes with subsystem
nodes_df.to_csv("factorybench/data/causrca_base/expert_graph/all_nodes_with_subsystem.csv", index=False)
print("\n✅ Saved: all_nodes_with_subsystem.csv")

# 7. show edges
edges_df = pd.read_csv("factorybench/data/causrca_base/expert_graph/all_edges.csv")

print("\n" + "="*60)
print("EDGES ANALYSIS")
print("="*60)
print("\nColumns available in EDGES:")
print(edges_df.columns.tolist())

print("\nFirst few edges:")
print(edges_df.head(10))

# 8. analyse edges finding source and target columns
edge_cols = edges_df.columns.tolist()
print(f"\nEdge DataFrame has {len(edge_cols)} columns: {edge_cols}")

source_col = None
target_col = None

source_names = ['source', 'from', 'u', 'source_id', 'node1', 'start']
target_names = ['target', 'to', 'v', 'target_id', 'node2', 'end']

for col in edge_cols:
    if col.lower() in source_names:
        source_col = col
    if col.lower() in target_names:
        target_col = col

if source_col and target_col:
    print(f"\n✅ Found edge columns: {source_col} → {target_col}")
    
    print("\n✅ Sample causal relationships:")
    for idx, edge in edges_df.head(20).iterrows():
        source_id = edge[source_col]
        target_id = edge[target_col]
        
        source_node = nodes_df[nodes_df['id'] == source_id]
        target_node = nodes_df[nodes_df['id'] == target_id]
        
        if len(source_node) > 0 and len(target_node) > 0:
            source_label = source_node.iloc[0]['label']
            target_label = target_node.iloc[0]['label']
            source_subsystem = source_node.iloc[0]['subsystem']
            target_subsystem = target_node.iloc[0]['subsystem']
            
            cross = "🔗" if source_subsystem != target_subsystem else "  "
            
            print(f"{cross} {source_label} ({source_subsystem}) → {target_label} ({target_subsystem})")
else:
    print(f"\n⚠️  Could not automatically detect source/target columns.")
    print(f"   Please inspect edges_df manually and update script.")
    print(f"\n   Available columns: {edge_cols}")
    print(f"\n   Sample row:")
    print(edges_df.iloc[0])

# final stats
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Total nodes: {len(nodes_df)}")
print(f"Total edges: {len(edges_df)}")
print(f"\nNodes by subsystem:")
for subsystem, count in subsystem_counts.items():
    print(f"  {subsystem}: {count}")

Nodes: 92
Edges: 104

Columns available in NODES:
['id', 'path', 'label', 'type', 'io', 'datatype', 'examples', 'comment']

First few rows:
   id                            path          label      type     io  \
0   1            Lathe/Coolant/Filter   CLF_A_700307     Alarm    NaN   
1   2            Lathe/Coolant/Filter  CLF_Filter_Ok  Variable  input   
2   3            Lathe/Coolant/Fleece     F_A_700313     Alarm    NaN   
3   4            Lathe/Coolant/Fleece    F_Filter_Ok  Variable  input   
4   5  Lathe/Coolant/HighPressurePump    HP_A_700304     Alarm    NaN   

  datatype                                      examples  \
0     bool  True (=alarm active), False(=alarm inactive)   
1     bool                                   True, False   
2     bool  True (=alarm active), False(=alarm inactive)   
3     bool                                   True, False   
4     bool  True (=alarm active), False(=alarm inactive)   

                                             comment  
0    

## Create component mapping structured
- Groups nodes into logical components (e.g., filters, pumps, sensors) and assigns them a type (actuator, container, sensor, etc.).
- Generates and saves the component_mapping.json file, which structures which variables and alarms belong to each physical component.

In [8]:
# component_mapping_from_expert_graph.py
import pandas as pd
import json

# Load nodes with subsystem
nodes_df = pd.read_csv("factorybench/data/causrca_base/expert_graph/all_nodes_with_subsystem.csv")

# Extract component name from path (third level)
# Example: "Lathe/Coolant/Filter" → "Filter"
nodes_df['component'] = nodes_df['path'].apply(
    lambda x: x.split('/')[2] if len(x.split('/')) > 2 else 'General'
)

# Create component mapping
component_mapping = {}

for subsystem in nodes_df['subsystem'].unique():
    subsystem_data = nodes_df[nodes_df['subsystem'] == subsystem]
    
    # Get unique components in this subsystem
    for component in subsystem_data['component'].unique():
        component_data = subsystem_data[subsystem_data['component'] == component]
        
        # Get variables and alarms
        variables = component_data[component_data['type'] == 'Variable'][['label', 'comment']].to_dict('records')
        alarms = component_data[component_data['type'] == 'Alarm'][['label', 'comment']].to_dict('records')
        
        # Create unique ID
        component_id = f"{subsystem}_{component}".replace(' ', '_')
        
        # Infer component type from name
        name_lower = component.lower()
        if any(word in name_lower for word in ['pump', 'motor']):
            comp_type = 'actuator'
        elif any(word in name_lower for word in ['filter', 'tank', 'reservoir', 'supply']):
            comp_type = 'container'
        elif any(word in name_lower for word in ['sensor', 'probe', 'measuring', 'monitoring']):
            comp_type = 'sensor'
        elif any(word in name_lower for word in ['valve', 'cover', 'lock']):
            comp_type = 'actuator'
        elif any(word in name_lower for word in ['spindle', 'tool', 'steady', 'rest']):
            comp_type = 'mechanical'
        elif any(word in name_lower for word in ['program', 'control']):
            comp_type = 'controller'
        else:
            comp_type = 'component'
        
        component_mapping[component_id] = {
            "id": component_id,
            "subsystem": subsystem,
            "name": component,
            "type": comp_type,
            "variables": variables,
            "alarms": alarms,
            "variable_count": len(variables),
            "alarm_count": len(alarms)
        }

# Save as JSON
with open("factorybench/data/component_mapping.json", "w") as f:
    json.dump(component_mapping, indent=2, fp=f)

# Print summary
print("✅ Component Mapping Created!")
print(f"\nTotal components: {len(component_mapping)}")

print("\n" + "="*60)
print("COMPONENTS BY SUBSYSTEM")
print("="*60)

for subsystem in sorted(nodes_df['subsystem'].unique()):
    subsystem_comps = [k for k, v in component_mapping.items() if v['subsystem'] == subsystem]
    print(f"\n{subsystem} ({len(subsystem_comps)} components):")
    
    for comp_id in sorted(subsystem_comps):
        comp = component_mapping[comp_id]
        print(f"  • {comp['name']:20} [{comp['type']:12}] - {comp['variable_count']} vars, {comp['alarm_count']} alarms")

# Create a simplified version for manual generation
simple_mapping = {}
for comp_id, comp in component_mapping.items():
    simple_mapping[comp_id] = {
        "subsystem": comp["subsystem"],
        "name": comp["name"],
        "type": comp["type"],
        "variables": [v['label'] for v in comp['variables']],
        "alarms": [a['label'] for a in comp['alarms']]
    }

with open("factorybench/data/component_mapping_simple.json", "w") as f:
    json.dump(simple_mapping, indent=2, fp=f)

print("\n✅ Also saved: component_mapping_simple.json (for manual generation)")

✅ Component Mapping Created!

Total components: 28

COMPONENTS BY SUBSYSTEM

Coolant (6 components):
  • Filter               [container   ] - 1 vars, 1 alarms
  • Fleece               [component   ] - 1 vars, 1 alarms
  • HighPressurePump     [actuator    ] - 2 vars, 1 alarms
  • LiftingTank          [container   ] - 2 vars, 1 alarms
  • LowPressurePump      [actuator    ] - 2 vars, 1 alarms
  • Tank                 [container   ] - 1 vars, 1 alarms

ExtractionUnit (1 components):
  • General              [component   ] - 2 vars, 0 alarms

Hydraulics (5 components):
  • General              [component   ] - 3 vars, 1 alarms
  • Hyd_Filter           [container   ] - 1 vars, 1 alarms
  • Hyd_Level            [component   ] - 1 vars, 2 alarms
  • Hyd_Pump             [actuator    ] - 3 vars, 1 alarms
  • Hyd_Temp             [component   ] - 2 vars, 2 alarms

Lubrication (1 components):
  • General              [component   ] - 2 vars, 0 alarms

MeasuringProbe (3 components):
  • Axis   

## Subsystem Documentation Generation
Consolidates information from the previous steps to create `subsystem_documentation.json`. This file aggregates all necessary technical data (components, variables, alarms, and internal causal relationships) for each subsystem to serve as context for generation.

In [9]:
import pandas as pd
import json

nodes_df = pd.read_csv("factorybench/data/causrca_base/expert_graph/all_nodes_with_subsystem.csv")
edges_df = pd.read_csv("factorybench/data/causrca_base/expert_graph/all_edges.csv")

with open("factorybench/data/component_mapping.json") as f:
    component_mapping = json.load(f)

# create documentation for each subsystem
docs = {}

for subsystem in nodes_df['subsystem'].unique():
    subsystem_nodes = nodes_df[nodes_df['subsystem'] == subsystem]
    
    # get components in this subsystem
    components = [c for c_id, c in component_mapping.items() if c['subsystem'] == subsystem]
    
    # get causal edges within this subsystem
    subsystem_node_ids = subsystem_nodes['id'].tolist()
    subsystem_edges = edges_df[
        (edges_df['source_id'].isin(subsystem_node_ids)) &
        (edges_df['target_id'].isin(subsystem_node_ids))
    ]
    
    docs[subsystem] = {
        "subsystem": subsystem,
        "node_count": len(subsystem_nodes),
        "component_count": len(components),
        "components": components,
        "internal_causal_edges": subsystem_edges[['source_label', 'target_label', 'comment']].to_dict('records'),
        "variables": subsystem_nodes[subsystem_nodes['type'] == 'Variable'][['label', 'comment']].to_dict('records'),
        "alarms": subsystem_nodes[subsystem_nodes['type'] == 'Alarm'][['label', 'comment']].to_dict('records')
    }

with open("factorybench/data/subsystem_documentation.json", "w") as f:
    json.dump(docs, indent=2, fp=f)

print("✅ Created: subsystem_documentation.json")
print("\nThis file contains everything needed to generate synthetic manuals:")
print("  - Component lists per subsystem")
print("  - Variables and their descriptions")
print("  - Alarms and their descriptions")
print("  - Internal causal relationships")

✅ Created: subsystem_documentation.json

This file contains everything needed to generate synthetic manuals:
  - Component lists per subsystem
  - Variables and their descriptions
  - Alarms and their descriptions
  - Internal causal relationships


## Generation of 19 synthetic troubleshooting manuals for causRCA fault scenarios
Iterates through 19 fault scenarios (experiments_info) from causRCA.
- Generating 19 synthetic troubleshooting manuals for causRCA fault scenarios.

- For each scenario, builds a detailed prompt with the subsystem context and the specific fault and generates a troubleshooting manual using **Azure OpenAI** in Markdown format and saves it to the `factorybench/manuals` folder.

In [ ]:

import os
import re
import json
import glob
from pathlib import Path
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv

load_dotenv()

AZURE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
API_KEY = os.getenv("AZURE_OPENAI_KEY")
DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT")
API_VERSION = "2025-01-01-preview"

COST_LIMIT_USD = 10.0
# pricing estimation (considering GPT-4 Turbo)
INPUT_PRICE_PER_1K = 0.01
OUTPUT_PRICE_PER_1K = 0.03

client = AzureOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=API_KEY,
    api_version=API_VERSION
)

total_cost = 0.0

def estimate_tokens(text):
    try:
        encoding = tiktoken.encoding_for_model("gpt-4")
        return len(encoding.encode(text))
    except Exception:
        return len(text) // 4

def update_cost(prompt_tokens, completion_tokens):
    global total_cost
    cost = (prompt_tokens / 1000 * INPUT_PRICE_PER_1K) + (completion_tokens / 1000 * OUTPUT_PRICE_PER_1K)
    total_cost += cost
    return cost

print("Loading system documentation...")
if os.path.exists("factorybench/data/component_mapping.json"):
    with open("factorybench/data/component_mapping.json") as f:
        component_mapping = json.load(f)
else:
    print("Warning: component_mapping.json not found")

if os.path.exists("factorybench/data/subsystem_documentation.json"):
    with open("factorybench/data/subsystem_documentation.json") as f:
        subsystem_docs = json.load(f)
else:
    print("Warning: subsystem_documentation.json not found")
    subsystem_docs = {}

# Load causRCA fault scenarios
print("Loading causRCA fault scenarios...")
experiments_info = []

base_path = "factorybench/data/causrca_base/dig_twin/"
subsystem_dirs = ["exp_coolant", "exp_hydraulics", "exp_probe"]

for subsystem_dir in subsystem_dirs:
    subsystem_path = os.path.join(base_path, subsystem_dir)
    
    if not os.path.exists(subsystem_path):
        print(f"⚠️  Warning: {subsystem_path} not found, skipping...")
        continue
    
    for exp_dir in os.listdir(subsystem_path):
        if not exp_dir.startswith("exp_"):
            continue
        
        exp_path = os.path.join(subsystem_path, exp_dir)
        desc_file = os.path.join(exp_path, f"{exp_dir}_description.json")
        
        if os.path.exists(desc_file):
            with open(desc_file) as f:
                description = json.load(f)
            
            # Load causes from run_1 (ground truth is same across runs)
            causes_file = os.path.join(exp_path, "run_1", "causes.json")
            causes = {}
            if os.path.exists(causes_file):
                with open(causes_file) as f:
                    causes = json.load(f)
            
            # Extract subsystem name (remove "exp_" prefix)
            subsystem = subsystem_dir.replace("exp_", "").capitalize()
            if subsystem == "Hydraulics":
                subsystem = "Hydraulics"  # Match with expert graph naming
            elif subsystem == "Coolant":
                subsystem = "Coolant"
            elif subsystem == "Probe":
                subsystem = "MeasuringProbe"  # Match with expert graph
            
            experiments_info.append({
                "exp_id": exp_dir,
                "subsystem": subsystem,
                "description": description,
                "causes": causes
            })

print(f"✅ Found {len(experiments_info)} fault scenarios\n")

# Create output directory
os.makedirs("factorybench/manuals", exist_ok=True)

def generate_manual(exp_info, subsystem_doc):
    """Generate a troubleshooting manual for a fault scenario using Azure OpenAI"""
    global total_cost
    
    if total_cost >= COST_LIMIT_USD:
        print("⛔ Cost limit reached. Stopping generation.")
        return None

    exp_id = exp_info["exp_id"]
    subsystem = exp_info["subsystem"]
    scenario = exp_info["description"].get("scenario", "Unknown scenario")
    diagnosis = exp_info["description"].get("diagnosis", "Unknown diagnosis")
    expected_alarms = exp_info["description"].get("expected_alarms", [])
    manipulated_vars = list(exp_info["causes"].keys())
    
    # Get relevant components
    components_list = [f"- {c['name']} ({c['type']})" for c in subsystem_doc['components']]
    
    # Get relevant variables (first 15 to avoid token limit)
    variables_list = [
        f"- {v['label']}: {v['comment'][:100]}..."
        for v in subsystem_doc['variables'][:15]
    ]
    
    # Get relevant alarms
    alarms_list = [
        f"- {a['label']}: {a['comment'][:100]}..."
        for a in subsystem_doc['alarms']
    ]
    
    # Get causal relationships (first 10)
    causal_chains = [
        f"- {edge['source_label']} → {edge['target_label']}: {edge['comment'][:80]}..."
        for edge in subsystem_doc['internal_causal_edges'][:10]
    ]
    
    prompt = f"""You are a senior technical writer creating troubleshooting documentation for a CNC vertical lathe manufacturing system.

# SYSTEM CONTEXT

**Machine Type:** CNC Vertical Lathe
**Subsystem:** {subsystem}
**Expert System:** This lathe has 92 monitored variables across 11 subsystems

# SUBSYSTEM COMPONENTS

The {subsystem} subsystem contains these components:
{chr(10).join(components_list)}

# MONITORED VARIABLES

Key variables in this subsystem:
{chr(10).join(variables_list)}

# SYSTEM ALARMS

Alarms that can be triggered:
{chr(10).join(alarms_list)}

# KNOWN CAUSAL RELATIONSHIPS

The expert system has documented these causal chains:
{chr(10).join(causal_chains)}

# FAULT SCENARIO TO DOCUMENT

**Fault ID:** {exp_id}
**Scenario Description:** {scenario}
**Diagnosis:** {diagnosis}
**Expected Alarms:** {', '.join(expected_alarms) if expected_alarms else 'Various alarms may trigger'}
**Root Cause (manipulated variables in digital twin):** {', '.join(manipulated_vars) if manipulated_vars else 'Multiple variables affected'}

# YOUR TASK

Create a comprehensive troubleshooting manual section for this specific fault. Structure it EXACTLY as follows:

## 1. Fault Overview
Provide a concise 2-3 sentence summary of what this fault is and why it matters.

## 2. System Impact
Describe the operational impact:
- What stops working?
- What safety systems activate?
- Production consequences?

## 3. Observable Symptoms

List symptoms operators would notice, organized by detection method:

**Via Alarms:**
- [Specific alarm codes and messages]

**Via Sensor Readings:**
- [Which variables show abnormal values]
- [Expected vs. actual ranges]

**Via Visual/Physical Inspection:**
- [What operators might see, hear, or smell]

## 4. Root Cause Analysis

### 4.1 Primary Root Cause
**Cause:** [Main failure mode based on manipulated variables]
**Mechanism:** [Explain HOW this causes the observed symptoms]
**Probability:** [High/Medium/Low based on context]

**Verification Steps:**
1. [Step-by-step procedure to confirm this is the cause]
2. [Include specific sensor checks with threshold values]
3. [Decision points: if X, then Y]

### 4.2 Secondary Causes
List 2-3 alternative causes that could produce similar symptoms:
- **Cause:** [Description]
  - **Likelihood:** [probability]
  - **Differentiating factors:** [How to distinguish from primary cause]

### 4.3 Causal Chain
Illustrate the fault propagation using this format:
```
[Root Event] → [Intermediate Effect 1] → [Intermediate Effect 2] → [Observable Symptom]
```

## 5. Diagnostic Procedure

Provide a decision-tree style diagnostic:

**Step 1: Initial Assessment**
- Action: [What to check first]
- Expected result: [Normal value/state]
- If abnormal: [Go to Step X]
- If normal: [Go to Step Y]

[Continue with Steps 2-5, creating a logical flow]

## 6. Remediation

### 6.1 Immediate Actions
**BEFORE any repair work:**
1. [Safety steps - power down, depressurize, etc.]
2. [Isolate the fault to prevent propagation]
3. [Document current state for analysis]

### 6.2 Repair Procedure

**Required:**
- Tools: [List specific tools]
- Parts: [Part numbers in format P/N: XXX-YYY-ZZZ]
- Personnel: [Skill level required]
- Estimated time: [X minutes/hours]

**Steps:**
1. [Detailed step with torque specs, clearances, etc.]
2. [Each step should be unambiguous]
3. [Include verification sub-steps]

### 6.3 Verification Tests
After repair:
1. [Test procedure to confirm fix]
2. [Expected sensor readings]
3. [Functional test sequence]

### 6.4 Return to Service
- [Calibration requirements]
- [Documentation to update]
- [Monitoring period recommendations]

## 7. Safety Warnings

⚠️ **CRITICAL SAFETY INFORMATION:**
- [Specific hazards for this fault]
- [PPE requirements]
- [Lock-out/tag-out procedures]
- [Pressure/temperature warnings]

## 8. Preventive Maintenance

**To prevent recurrence:**
- [Inspection intervals]
- [Wear indicators to monitor]
- [Maintenance schedule recommendations]

## 9. Related Faults

**This fault can cause:**
- [Downstream faults that may occur]

**This fault can be caused by:**
- [Upstream faults that may trigger this]

**Similar symptoms but different causes:**
- [Faults that operators might confuse with this one]

## 10. Technical Notes

- Document revision: 1.0
- Last updated: February 2026
- Based on: Digital twin experiment {exp_id}
- Expert graph node references: {', '.join(manipulated_vars[:5])}

---

# IMPORTANT GUIDELINES:

1. **Be specific:** Use actual variable names from the system (e.g., "HP_Pump_Ok", "CLF_A_700307")
2. **Be technical but clear:** Write for trained technicians, not engineers
3. **Be actionable:** Every step should be executable without ambiguity
4. **Be realistic:** Part numbers should follow format P/N: [SUBSYS]-[COMP]-[NUM] (e.g., P/N: COOL-PUMP-HP-001)
5. **Be consistent:** Use terminology from the variable descriptions provided
6. **Reference the causal chains** provided above where relevant

Generate a professional, publication-ready troubleshooting manual section now."""

    print(f"  Generating manual for {exp_id}...")
    
    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        manual_text = response.choices[0].message.content
        
        # Track cost
        usage = response.usage
        cost = update_cost(usage.prompt_tokens, usage.completion_tokens)
        print(f"  💲 Cost: ${cost:.4f} | Total Session: ${total_cost:.4f}")
        
        return manual_text
    
    except Exception as e:
        print(f"  ❌ Error generating manual for {exp_id}: {str(e)}")
        return None

# Generate manuals for all fault scenarios
print("="*60)
print("GENERATING SYNTHETIC MANUALS (AZURE OPENAI)")
print("="*60)
print(f"Spending Limit: ${COST_LIMIT_USD}")

generated_count = 0
failed_count = 0

for i, exp_info in enumerate(experiments_info):
    if total_cost >= COST_LIMIT_USD:
         print(f"\n⛔ Spending limit reached (${COST_LIMIT_USD}). Stopping.")
         break

    exp_id = exp_info["exp_id"]
    subsystem = exp_info["subsystem"]
    diagnosis = exp_info["description"].get("diagnoses")[0]['name'] if exp_info["description"].get("diagnoses") else "unknown"
    if diagnosis != "unknown":
        clean = re.sub(r'[^a-zA-Z0-9_\- ]', '', diagnosis)
        diagnosis = clean[:40].strip().replace(' ', '_')
    
    print(f"\n[{i+1}/{len(experiments_info)}] {exp_id} ({subsystem})")
    
    # Get subsystem documentation
    if subsystem not in subsystem_docs:
        print(f"  ⚠️  Subsystem '{subsystem}' not found in documentation, skipping...")
        failed_count += 1
        continue
    
    subsystem_doc = subsystem_docs[subsystem]
    
    # Generate manuals
    manual_text = generate_manual(exp_info, subsystem_doc)
    
    if manual_text:
        filename = f"{exp_id}_{subsystem}_{diagnosis.replace(' ', '_').replace('/', '_')}.md"
        filepath = os.path.join("factorybench/manuals", filename)
        
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(f"# Troubleshooting Manual: {diagnosis}\n\n")
            f.write(f"**Fault ID:** {exp_id}\n")
            f.write(f"**Subsystem:** {subsystem}\n")
            f.write(f"**Scenario:** {exp_info['description'].get('scenario', 'N/A')}\n\n")
            f.write("---\n\n")
            f.write(manual_text)
        
        print(f"  ✅ Saved: {filename}")
        generated_count += 1
    else:
        failed_count += 1

print(f"\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Total fault scenarios: {len(experiments_info)}")
print(f"Manuals generated: {generated_count}")
print(f"Failed: {failed_count}")
print(f"Total Cost: ${total_cost:.4f}")
print(f"\n✅ All manuals saved in: factorybench/manuals/")


Loading system documentation...
Loading causRCA fault scenarios...
✅ Found 19 fault scenarios

GENERATING SYNTHETIC MANUALS (AZURE OPENAI)
Spending Limit: $10.0

[1/19] exp_22 (Coolant)
  Generating manual for exp_22...
  💲 Cost: $0.0768 | Total Session: $0.0768
  ✅ Saved: exp_22_Coolant_Leakage_in_coolant_system.md

[2/19] exp_27 (Coolant)
  Generating manual for exp_27...
  💲 Cost: $0.0728 | Total Session: $0.1496
  ✅ Saved: exp_27_Coolant_Leakage_in_coolant_system.md

[3/19] exp_28 (Coolant)
  Generating manual for exp_28...
  💲 Cost: $0.0777 | Total Session: $0.2273
  ✅ Saved: exp_28_Coolant_Highly_Contaminated_Coolant.md

[4/19] exp_35 (Coolant)
  Generating manual for exp_35...
  💲 Cost: $0.0772 | Total Session: $0.3045
  ✅ Saved: exp_35_Coolant_Highly_Contaminated_Coolant.md

[5/19] exp_10 (Hydraulics)
  Generating manual for exp_10...
  💲 Cost: $0.0835 | Total Session: $0.3880
  ✅ Saved: exp_10_Hydraulics_Hydraulic_oil_cooler_defective.md

[6/19] exp_12 (Hydraulics)
  Generatin

Example of experiment information:

```json
{'exp_id': 'exp_6', 'subsystem': 'MeasuringProbe',
    'description': {'exp_id': 'exp_6', 'group': 'probe', 'scenario': {
        'init': 'Measuring probe axis (MPA) is at initial position (IP = true, m2IP = true, WP = false, m2WP = false)',
        'action': 'The control system commands a move to the working position (m2WP = true -> m2IP = false)',
        'expected_result': 'IP switches to false as the MPA leaves the initial position, and WP becomes true within a few seconds (MPA reaches and is recognized at working position)',
        'instead_failure': 'IP remains true - no movement has occurred / the axis is stuck'},
        'manipulatedVars': ['MPA_InitPos', 'MPA_WorkPos'],
        'alarms': ['MPA_A_701124'],
        'diagnoses': [{'id': 'diag_3', 'name': 'MPA is stuck -> does not move besides being commanded to.', 'comment': 'MPA is stuck in initial position / move to working  position command is not recognized.', 'variablesStates': {'MPA_InitPos': True, 'MPA_toInitPos': False, 'MPA_toWorkPos': True, 'MPA_WorkPos': False}}]
    },
    'causes': {'cause_start_at': 79.603, 'alarms_detected_at': 100.115, 'diagnosis_at': 123.99, 'cause_end_at': 147.865, 'alarms_resolved_at': 150.389}}
```



```json
{'exp_id': 'exp_6', 'subsystem': 'MeasuringProbe',
    'description': {'exp_id': 'exp_6', 'group': 'probe', 'scenario': {
        'init': 'Measuring probe axis (MPA) is at initial position (IP = true, m2IP = true, WP = false, m2WP = false)',
        'action': 'The control system commands a move to the working position (m2WP = true -> m2IP = false)',
        'expected_result': 'IP switches to false as the MPA leaves the initial position, and WP becomes true within a few seconds (MPA reaches and is recognized at working position)',
        'instead_failure': 'IP remains true - no movement has occurred / the axis is stuck'},
        'manipulatedVars': ['MPA_InitPos', 'MPA_WorkPos'],
        'alarms': ['MPA_A_701124'],
        'diagnoses': [{'id': 'diag_3', 'name': 'MPA is stuck -> does not move besides being commanded to.', 'comment': 'MPA is stuck in initial position / move to working  position command is not recognized.', 'variablesStates': {'MPA_InitPos': True, 'MPA_toInitPos': False, 'MPA_toWorkPos': True, 'MPA_WorkPos': False}}]
    },
    'causes': {'cause_start_at': 79.603, 'alarms_detected_at': 100.115, 'diagnosis_at': 123.99, 'cause_end_at': 147.865, 'alarms_resolved_at': 150.389}}
```



# GraphRAG with FAISS
Implementation of a Retrieval Augmented Generation system using the generated manuals and FAISS.
This section indexes the markdown manuals we just created and allows for semantic query retrieval.

In [17]:
import faiss
import numpy as np
import glob
import os
from typing import List, Dict

# Configuration
MANUALS_DIR = "factorybench/manuals"
EMBEDDING_MODEL = "text-embedding-3-small" 

def get_embedding(text: str) -> List[float]:
    try:
        response = client.embeddings.create(
            input=text,
            model=EMBEDDING_MODEL
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embedding: {e}")
        # Return a zero vector of size 384 (common for text-embedding-3-small) if failure
        return [0.0] * 384

In [12]:
# Load and Chunk Manuals
def load_and_chunk_manuals(directory: str, chunk_size: int = 1000, overlap: int = 100) -> List[Dict]:
    chunks = []
    
    files = glob.glob(f"{directory}/*.md")
    print(f"Found {len(files)} manuals by globbing {directory}/*.md")
    
    for filepath in files:
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                content = f.read()
                
            filename = os.path.basename(filepath)
            
            for i in range(0, len(content), chunk_size - overlap):
                chunk_text = content[i:i + chunk_size]
                if len(chunk_text) < 50: continue
                
                chunks.append({
                    "text": chunk_text,
                    "source": filename,
                    "start": i
                })
        except Exception as e:
            print(f"Error reading {filepath}: {e}")
            
    return chunks

chunks = load_and_chunk_manuals(MANUALS_DIR)
print(f"Created {len(chunks)} chunks.")

Found 19 manuals by globbing factorybench/manuals/*.md
Created 199 chunks.


In [ ]:
# Embed and Index
print("Generating embeddings...\n")
embeddings = []
valid_chunks = []

BATCH_SIZE = 10
for i in range(0, len(chunks), BATCH_SIZE):
    batch = chunks[i:i+BATCH_SIZE]
    for chunk in batch:
        emb = get_embedding(chunk["text"])
        if np.any(emb):
            embeddings.append(emb)
            valid_chunks.append(chunk)
            
    if i % 50 == 0:
        print(f"Processed {i}/{len(chunks)}...")

if embeddings:
    dimension = len(embeddings[0])
    db_vectors = np.array(embeddings).astype('float32')

    # Build FAISS index
    index = faiss.IndexFlatL2(dimension)
    index.add(db_vectors)
    print(f"\nIndexed {index.ntotal} vectors in FAISS.")
else:
    print("No embeddings generated. Check your API key or connection.")

Generating embeddings (this may take a while)...

Error getting embedding: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
Error getting embedding: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
Error getting embedding: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
Error getting embedding: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wa

In [14]:
# Query Function (GraphRAG)
def graph_rag_query(query: str, k: int = 3):
    if 'index' not in globals():
        print("Index not created.")
        return

    # Embed query
    q_emb = np.array([get_embedding(query)]).astype('float32')
    
    # Search FAISS
    distances, indices = index.search(q_emb, k)
    
    retrieved_contexts = []
    for idx in indices[0]:
        if idx != -1 and idx < len(valid_chunks):
            retrieved_contexts.append(valid_chunks[idx])
            
    
    context_text = "\n\n---\n\n".join([f"Source: {c['source']}\n{c['text']}" for c in retrieved_contexts])
    
    system_prompt ="""
        You are an expert troubleshooter for a CNC lathe. 
        Use the provided Context from the repair manuals to answer the user's question. 
        If the manuals mention specific causal chains or graph nodes, highlight them.
    """

    user_prompt = f"""Question: {query}

Context:
{context_text}

Answer:"""

    response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content


In [15]:
# Example tests
queries = [
    "What causes the coolant filter to be clogged?",
    "How do I fix a hydraulic pump failure?"
]

for q in queries:
    print(f"\n{'='*40}\nQuery: {q}\n{'='*40}")
    try:
        ans = graph_rag_query(q)
        print(ans)
    except Exception as e:
        print(f"Error: {e}")


Query: What causes the coolant filter to be clogged?
Index not created.
None

Query: How do I fix a hydraulic pump failure?
Index not created.
None
